# Decorators

Python decorators are a powerful concept that allow you to "wrap" a function with another function. A function can be decorated many times, if desired.

* [functools](https://docs.python.org/3/library/functools.html) — Higher-order functions and operations on callable objects
* [How to use functools.wraps](https://www.blog.pythonlibrary.org/2016/02/17/python-functools-wraps/)
* [inspect — Inspect live objects](https://docs.python.org/3/library/inspect.html), [inspect.getcallargs()](https://docs.python.org/3/library/inspect.html#inspect.getcallargs)

## Functions That Return Other Functions

* When you define a function in python, that function becomes an object.
* A _decorator_ is a function that takes another function as an argument and replace it with a new, modified function.

In [1]:
def apply(func, arg):
    return func(arg)

In [2]:
apply(len, "hello")

5

In [7]:
def add(num):
    def adder(x):
        return num + x
    return adder

In [8]:
func = add(10)

In [9]:
func(5)

15

In [10]:
add(1)(10)

11

In [11]:
def string_required(f):
    def validate(arg):
        if type(arg) != str:
            raise ValueError(f'{repr(arg)} is not a string')
        return f(arg)
    return validate

In [12]:
string_required(len)("limited len")

11

In [13]:
string_required(len)([0, 1, 2, 3])

ValueError: [0, 1, 2, 3] is not a string

### One more example

In [2]:
_functions = {}

def register(func):
    global _functions
    _functions[func.__name__] = func
    return func

@register
def foo():
    return 'bar'

_functions['foo']

<function __main__.foo()>

## Decorators Are A Syntax Sugar

In [15]:
@string_required
def greeting(name):
    print(f'Hi, {name}!')

In [16]:
greeting('Andrey')

Hi, Andrey!


In [17]:
greeting(1)

ValueError: 1 is not a string

In [18]:
@string_required
class Greeter:
    def __init__(self, name):
        self.name = name
        
    def say(self):
        print(f'Hi, {self.name}!')

In [19]:
Greeter('Andrey').say()

Hi, Andrey!


In [20]:
Greeter(('Andrey', 'Maria', 'Mark')).say()

ValueError: ('Andrey', 'Maria', 'Mark') is not a string

### One more example

Warning: this naive approach has some major drawbacks! Try to implement stacking decorators in this way.

In [4]:
def admin_required(f):
    def wrapper(*args, **kwargs):
        if kwargs.get('username') != 'admin':
            raise Exception('This user is not allowed to log in')
        return f(*args, **kwargs)
    
    return wrapper

class SiteView:
    @admin_required
    def admin_page(self, username):
        return 'Admin area'

Tip: Check the `inspect` module. It's better to use its functions rather checking whether function parameters are a positional or a keyword arguments.

## Injecting Logic Into Class Methods

_Class decorators_ work in the same way as function decorators, but they act on classes rather than functions.

In [5]:
import uuid

def set_class_name_and_id(klass):
    klass.name = str(klass)
    klass.random_id = uuid.uuid4()
    return klass

@set_class_name_and_id
class SomeClass:
    pass

SomeClass().random_id

UUID('de4708e1-3a35-4d76-8611-fb97c421e870')

In [21]:
def barking(cls):
    for name in cls.__dict__:
        if name.startswith('__'):
            continue
        
        func = getattr(cls, name)

        def woofer(*args, **kwargs):
            print('Woof')
            return func(*args, **kwargs)

        setattr(cls, name, woofer)
        
    return cls

In [22]:
@barking
class Dog1:
    def shout(self):
        print("I'm a dog!")

In [23]:
d = Dog1()

In [24]:
d

In [27]:
d.shout()

Woof
I'm a dog!


Class decorators are useful for wrapping a function that's storing a state.

In [10]:
class CountCalls:
    def __init__(self, f):
        self.f = f
        self.called = 0
        
    def __call__(self, *args, **kwargs):
        self.called += 1
        return self.f(*args, **kwargs)
    
@CountCalls
def greeting():
    print("Hi")
    
greeting.called

0

In [11]:
greeting()
greeting()
greeting()

Hi
Hi
Hi


In [12]:
greeting.called

3

## functools.update_wrapper()

A decorator replaces the original function with a new one built on the fly. This new function lacks many of the attributes of the original function, such as its docstring and its name. The `functools.update_wrapper()` function solves this problem. It copies the attributes from the original function that were lost to the wrapper itself.

```python
def foobar(username='someone'):
    pass

foobar = functools.update_wrapper(is_admin, foobar)
foobar.__name__
foobar.__doc__
```

### wraps() – decorator for decorators

It can get tedious to use `update_wrapper()` manually when creating decorators, so functools provides a decorator for decorators called `wraps`. With `functools.wrap()`, the decorator function that returns the `wrapper()` function takes care of copying the docstring, name function, and other information from the function `f` passed as argument.

## Decorators Can Return Something That Is Not A Function

The decorator has the ability to swallow the function, or return something that is not a function, if it wanted.

In [28]:
def stupid(cls):
    class Null:
        pass
    return Null

In [29]:
@stupid
class Something:
    def method1(self):
        pass
    def method2(self):
        pass

In [30]:
Something

__main__.stupid.<locals>.Null

In [31]:
Something.method1()

AttributeError: type object 'Null' has no attribute 'method1'